In [1]:
import re
import json
import pickle
import pandas as pd
from glob import glob
import torch
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize

In [2]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli', force_reload=True)
roberta.eval();

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /tmp/xdg-cache/torch/hub/master.zip
/home/tamittal/.local/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp


/home/tamittal/.local/lib/python3.7/site-packages/torch/utils/cpp_extension.py:369: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))


cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/fairseq
creating build/temp.linux-x86_64-3.7/fairseq/clib
creating build/temp.linux-x86_64-3.7/fairseq/clib/libbleu
gcc -pthread -B /opt/conda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/conda/include/python3.7m -c fairseq/clib/libbleu/libbleu.cpp -o build/temp.linux-x86_64-3.7/fairseq/clib/libbleu/libbleu.o -std=c++11 -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE="_gcc" -DPYBIND11_STDLIB="_libstdcpp" -DPYBIND11_BUILD_ABI="_cxxabi1011" -DTORCH_EXTENSION_NAME=libbleu -D_GLIBCXX_USE_CXX11_ABI=0
gcc -pthread -B /opt/conda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/conda/include/python3.7m -c fairseq/clib/libbleu/module.cpp -o bu

In [4]:
index = json.load(open("../data/wiki-index.json"))
len(index)

5416536

In [3]:
df = pd.DataFrame([json.loads(l.strip()) for l in open("../data/train.jsonl").readlines()]).set_index("id").sort_index()
print("# of claims in training data:", len(df))
df.head()

# of claims in training data: 145449


,verifiable,label,claim,evidence
id,,,,
3,VERIFIABLE,SUPPORTS,Chris Hemsworth appeared in A Perfect Getaway.,"[[[15732, 19585, Chris_Hemsworth, 2]]]"
4,NOT VERIFIABLE,NOT ENOUGH INFO,Chris Hemsworth disappeared in A Perfect Getaway.,"[[[15230, None, None, None]]]"
7,VERIFIABLE,SUPPORTS,Roald Dahl is a writer.,"[[[15234, 18987, Roald_Dahl, 0]]]"
8,VERIFIABLE,REFUTES,Roald Dahl is a governor.,"[[[15233, 18986, Roald_Dahl, 0]]]"
9,VERIFIABLE,SUPPORTS,Ireland has relatively low-lying mountains.,"[[[15235, 18988, Ireland, 10]]]"


In [60]:
sample = df[df["label"] != "NOT ENOUGH INFO"].groupby("label").sample(50)
sample.head()

,verifiable,label,claim,evidence
id,,,,
97054,VERIFIABLE,REFUTES,David Jones (video game developer) is uninvolv...,"[[[114873, 128600, David_Jones_-LRB-video_game..."
37796,VERIFIABLE,REFUTES,Bangladesh is a continent.,"[[[53972, 63960, Bangladesh, 1]]]"
124944,VERIFIABLE,REFUTES,Seven Days in May stars Robert Pattinson.,"[[[146600, 161586, Seven_Days_in_May, 0], [146..."
44737,VERIFIABLE,REFUTES,Maynard James Keenan has no involvement in the...,"[[[61106, 71573, Maynard_James_Keenan, 14]], [..."
176954,VERIFIABLE,REFUTES,Underworld is a religion.,"[[[205056, 213689, Underworld_-LRB-2003_film-R..."


In [61]:
ex = [[[142170, 157393, "Mexico", 3]]]
def get_sent(x):
    try:
        return re.split(r"\n", dict(json.load(open("../data/wiki-indexed/%s" % index[x[0][0][2]])))[x[0][0][2]]["lines"])[x[0][0][3]][len(str(x[0][0][3])):]
    except:
        print(x)
sample["sentence"] = sample["evidence"].apply(get_sent)
sample["sentence"] = sample["sentence"].str.replace(r"\t", " ").str.strip()
sample = sample[~sample["sentence"].isnull()]
sample.head()

[[[206389, 214749, 'Sophia_Abrahão', 0]]]
[[[151868, 166626, 'Citadelle_Laferrière', 0]]]


,verifiable,label,claim,evidence,sentence
id,,,,,
97054,VERIFIABLE,REFUTES,David Jones (video game developer) is uninvolv...,"[[[114873, 128600, David_Jones_-LRB-video_game...",\tDavid Scott Jones -LRB- born October 1965 -R...
37796,VERIFIABLE,REFUTES,Bangladesh is a continent.,"[[[53972, 63960, Bangladesh, 1]]]","\t`` The country of Bengal '' -RRB- , official..."
124944,VERIFIABLE,REFUTES,Seven Days in May stars Robert Pattinson.,"[[[146600, 161586, Seven_Days_in_May, 0], [146...",\tSeven Days in May is a 1964 American politic...
44737,VERIFIABLE,REFUTES,Maynard James Keenan has no involvement in the...,"[[[61106, 71573, Maynard_James_Keenan, 14]], [...","\tAmong other business ventures , he is a part..."
176954,VERIFIABLE,REFUTES,Underworld is a religion.,"[[[205056, 213689, Underworld_-LRB-2003_film-R...",\tUnderworld is a 2003 action horror film dire...


In [88]:
sample.head()

,verifiable,label,claim,evidence,sentence,preds
id,,,,,,
97054,VERIFIABLE,REFUTES,David Jones (video game developer) is uninvolv...,"[[[114873, 128600, David_Jones_-LRB-video_game...",David Scott Jones -LRB- born October 1965 -RRB...,REFUTES
37796,VERIFIABLE,REFUTES,Bangladesh is a continent.,"[[[53972, 63960, Bangladesh, 1]]]","`` The country of Bengal '' -RRB- , officially...",SUPPORTS
124944,VERIFIABLE,REFUTES,Seven Days in May stars Robert Pattinson.,"[[[146600, 161586, Seven_Days_in_May, 0], [146...",Seven Days in May is a 1964 American political...,NEUTRAL
44737,VERIFIABLE,REFUTES,Maynard James Keenan has no involvement in the...,"[[[61106, 71573, Maynard_James_Keenan, 14]], [...","Among other business ventures , he is a partne...",REFUTES
176954,VERIFIABLE,REFUTES,Underworld is a religion.,"[[[205056, 213689, Underworld_-LRB-2003_film-R...",Underworld is a 2003 action horror film direct...,NEUTRAL


In [89]:
label_map = {0:"REFUTES", 1:"NEUTRAL", 2:"SUPPORTS", -1:"ERROR"}

In [95]:
row = sample.iloc[2]
probs = roberta.predict('mnli', roberta.encode(row["claim"], row["sentence"]))

In [91]:
def get_pred(row):
    try:
        return roberta.predict('mnli', roberta.encode(row["claim"], row["sentence"])).argmax().item()
    except:
        print(row["claim"])
        print(row["sentence"])
        return -1
sample["preds"] = sample.apply(get_pred, axis=1)
sample.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,verifiable,label,claim,evidence,sentence,preds
id,,,,,,
97054,VERIFIABLE,REFUTES,David Jones (video game developer) is uninvolv...,"[[[114873, 128600, David_Jones_-LRB-video_game...",David Scott Jones -LRB- born October 1965 -RRB...,0
37796,VERIFIABLE,REFUTES,Bangladesh is a continent.,"[[[53972, 63960, Bangladesh, 1]]]","`` The country of Bengal '' -RRB- , officially...",2
124944,VERIFIABLE,REFUTES,Seven Days in May stars Robert Pattinson.,"[[[146600, 161586, Seven_Days_in_May, 0], [146...",Seven Days in May is a 1964 American political...,1
44737,VERIFIABLE,REFUTES,Maynard James Keenan has no involvement in the...,"[[[61106, 71573, Maynard_James_Keenan, 14]], [...","Among other business ventures , he is a partne...",0
176954,VERIFIABLE,REFUTES,Underworld is a religion.,"[[[205056, 213689, Underworld_-LRB-2003_film-R...",Underworld is a 2003 action horror film direct...,1


In [92]:
sample["preds"] = sample["preds"].apply(lambda x: label_map[x])
sample.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,verifiable,label,claim,evidence,sentence,preds
id,,,,,,
97054,VERIFIABLE,REFUTES,David Jones (video game developer) is uninvolv...,"[[[114873, 128600, David_Jones_-LRB-video_game...",David Scott Jones -LRB- born October 1965 -RRB...,REFUTES
37796,VERIFIABLE,REFUTES,Bangladesh is a continent.,"[[[53972, 63960, Bangladesh, 1]]]","`` The country of Bengal '' -RRB- , officially...",SUPPORTS
124944,VERIFIABLE,REFUTES,Seven Days in May stars Robert Pattinson.,"[[[146600, 161586, Seven_Days_in_May, 0], [146...",Seven Days in May is a 1964 American political...,NEUTRAL
44737,VERIFIABLE,REFUTES,Maynard James Keenan has no involvement in the...,"[[[61106, 71573, Maynard_James_Keenan, 14]], [...","Among other business ventures , he is a partne...",REFUTES
176954,VERIFIABLE,REFUTES,Underworld is a religion.,"[[[205056, 213689, Underworld_-LRB-2003_film-R...",Underworld is a 2003 action horror film direct...,NEUTRAL


In [93]:
(sample["label"] == sample["preds"]).mean()

0.3877551020408163

In [ ]:
torch.cuda.get_device_name(0)

In [5]:
torch.cuda.is_available()

False